In [1]:
import os
import requests
import yaml
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool, tool
from langchain_core.tools import ToolException
from typing import List, Optional, Dict, Any
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

In [2]:
class MyAgent:
    def __init__(self, tools, custom_prompt: str = None):
        self.llm = AzureChatOpenAI(
            model="gpt-4o",
        )
        self.tools = tools
        self.agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent = create_tool_calling_agent(
            llm=self.llm,
            tools=self.enhance_agent_tools(tools),
            prompt=ChatPromptTemplate.from_messages([
                ("system", custom_prompt if custom_prompt else "You are an experienced assistant. You are able to perform any task asked by the user through all the tools you have access to. Your goal is to complete the user's task by using the tools you have access to. This is a core instruction: ALWAYS ALWAYS ALWAYS USE A STRING AS ID"),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]),
        )
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, memory=self.agent_memory, verbose=True)

    def myinvoke(self, input: str):
        if not isinstance(input, str):
            raise ValueError("Input must be a string")
        result = self.executor.invoke({"input": input})
        print(result)
        print("\n==================================================")
        print("--> Agent Response:")
        print(result['output'])
        print("\n==================================================")

    def try_except_tool(self, func):
        try:
            return func
        except ToolException as e:
            print(f"Error: {e}")

    def enhance_agent_tools(self, agent_tools: List):
        enhanced_agent_tools = [
            StructuredTool(
                name=tool.name,
                description=tool.description,
                args_schema=tool.args_schema,
                func=self.try_except_tool(tool._run),
                handle_tool_error=True,
                handle_validation_error=True
            ) for tool in agent_tools
        ]
        return enhanced_agent_tools

In [3]:
def list_fabric_patterns():
    """
    List all available fabric patterns from GitHub
    """
    url = "https://api.github.com/repos/danielmiessler/fabric/contents/patterns"
    response = requests.get(url)
    if response.status_code == 200:
        patterns = [item['name'] for item in response.json() if item['type'] == 'dir']
        return patterns
    else:
        raise Exception(f"Failed to fetch patterns: {response.status_code}")

@tool
def use_fabric_pattern(pattern_name: str):
    """
    Use a fabric pattern to get instructions for each pattern's specific use case
    """
    url = f"https://raw.githubusercontent.com/danielmiessler/fabric/main/patterns/{pattern_name}/system.md"
    response = requests.get(url)
    if response.status_code == 200:
        pattern_str = response.text
        pattern_str = pattern_str.replace("{", "[[").replace("}", "]]")
        return pattern_str
    else:
        raise Exception(f"Failed to fetch pattern: {response.status_code}")

In [4]:
prompt = f"""You are a loyal, tireless assistant, who's been granted access to Fabric AI's 
entire pattern collection.Fabric AI is at its core a collection of great prompts built 
collaboratively by the community. These prompts in the context of Fabric are called 
'patterns'. Each pattern is a collection of instructions that can be used to perform a 
specific task. You can ask me for a list of the available patterns or use a specific 
pattern to get instructions for a specific task"
List all available fabric patterns: {list_fabric_patterns()}"""


agent = MyAgent([use_fabric_pattern], prompt)

In [5]:
agent.myinvoke("List all available fabric patterns")



> Entering new AgentExecutor chain...
Here are all the available Fabric patterns:

1. agility_story
2. ai
3. analyze_answers
4. analyze_cfp_submission
5. analyze_claims
6. analyze_debate
7. analyze_email_headers
8. analyze_incident
9. analyze_interviewer_techniques
10. analyze_logs
11. analyze_malware
12. analyze_paper
13. analyze_patent
14. analyze_personality
15. analyze_presentation
16. analyze_prose
17. analyze_prose_json
18. analyze_prose_pinker
19. analyze_sales_call
20. analyze_spiritual_text
21. analyze_tech_impact
22. analyze_threat_report
23. analyze_threat_report_trends
24. answer_interview_question
25. ask_secure_by_design_questions
26. capture_thinkers_work
27. check_agreement
28. clean_text
29. coding_master
30. compare_and_contrast
31. create_5_sentence_summary
32. create_academic_paper
33. create_ai_jobs_analysis
34. create_aphorisms
35. create_art_prompt
36. create_better_frame
37. create_coding_project
38. create_command
39. create_cyber_summary
40. create_formal_em